In [1]:
# !pip install dataset
# !pip install setfit

# # # skew towards false postive is good

In [2]:
import pandas as pd
import pyarrow as pa
from sklearn.model_selection import train_test_split
from datasets import Dataset


# Specify the path to your Excel file
# file_path = '/content/companydata.xlsx'

# Read the Excel file into a DataFrame
# df = pd.read_excel(file_path)

df = pd.read_csv('/content/combined_df.csv')
df.rename(columns={'Company Li Description': 'text', 'relevant': 'label'}, inplace=True)
df = df.drop('Unnamed: 0', axis=1)


# Get only 20 instances of each label in the training set
df = df.groupby('label').head(50).reset_index(drop=True)


# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)

# Split the temporary set into testing and validation sets
test_df, val_df = train_test_split(temp_df, test_size=0.5, random_state=42)


train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
eval_dataset = Dataset.from_pandas(val_df)



In [3]:
# df['label'].value_counts()

In [4]:
train_dataset

Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 91
})

In [5]:
# from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
import os
os.environ['WANDB_DISABLED'] = 'true'


# Load a SetFit model from Hub
model = SetFitModel.from_pretrained(
    # "sentence-transformers/paraphrase-mpnet-base-v2",
    # "tyzp-INC/few-shot-multilingual-e5-large-xnli",
    # "thenlper/gte_small",
    # "moshew/bge-small-en-v1.5_setfit-sst2-english",
    "BAAI/bge-small-en-v1.5",
    # labels=["negative", "positive"],
    labels=[0,1,2],
)

args = TrainingArguments(
    batch_size=12,
    num_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    column_mapping={"text": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate(test_dataset)
print(metrics)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Applying column mapping to the training dataset
Applying column mapping to the evaluation dat

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 5684
  Batch size = 12
  Num epochs = 5
  Total optimization steps = 2370


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.196000,0.000018
2,No log,No log,0.210200,0.000013
3,No log,No log,0.217400,0.000009
4,No log,No log,0.222000,0.000004
5,No log,No log,0.225300,0.000000


  0%|          | 0/54 [00:00<?, ?it/s]

...

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 474.
Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.5806451612903226}


In [6]:
preds = model(["Cohesive helps small business owners save time and money. We offer technology-enabled virtual assistance services.  Check us out at www.cohesiveapp.com!"])
preds

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['1'], dtype='<U2')

In [20]:
test_sentences =test_df['text'].tolist()
test_sentences_y =test_df['label'].tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
test_sentences_y

['1',
 '2',
 '0',
 '0',
 '1',
 '0',
 '0',
 '2',
 '1',
 '2',
 '0',
 '2',
 '1',
 '2',
 '2',
 '1',
 '1',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '2',
 '0',
 '2',
 '2',
 '0',
 '2',
 '1',
 '1']

In [22]:
model(test_sentences).astype(int)

array([0, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1,
       0, 2, 1, 2, 2, 0, 1, 1, 1])

In [23]:
import torch

def f1_score_pytorch(predictions, targets):
    predictions = torch.tensor(predictions)
    targets = torch.tensor(targets)

    # Calculate true positives, false positives, and false negatives
    tp = torch.sum(predictions * targets)
    fp = torch.sum(predictions * (1 - targets))
    fn = torch.sum((1 - predictions) * targets)

    # Calculate precision and recall
    precision = tp / (tp + fp + 1e-10)
    recall = tp / (tp + fn + 1e-10)

    # Calculate F1 score
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)

    return f1.item(),precision,recall

In [24]:
import torch
from torcheval.metrics.functional import multiclass_f1_score
input = torch.tensor(list(model(test_sentences).astype(int)))
target = torch.tensor([int(x) for x in test_sentences_y])
multiclass_f1_score(input, target, num_classes=3, average=None)

tensor([0.6957, 0.4545, 0.5882])